<a href="https://colab.research.google.com/github/zzyuzzz/123/blob/master/5645646464564564686.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch 
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Compose, Resize
from torchvision import datasets

In [2]:
training_data = datasets.WIDERFace(
    root='data',
    split='train',
    transform=ToTensor(),
    download=True
)

testing_data = datasets.WIDERFace(
    root='data',
    split='test',
    transform=ToTensor(),
    download=True
)

1465602149it [00:24, 59936388.53it/s] 
362752168it [00:04, 76872009.92it/s]
1844140520it [00:27, 66912998.60it/s]


100%|██████████| 3591642/3591642 [00:00<00:00, 18843856.48it/s]


Extracting data/widerface/wider_face_split.zip to data/widerface
Files already downloaded and verified


In [3]:
testing_data[0]

(tensor([[[0.0275, 0.0392, 0.0627,  ..., 0.0471, 0.0471, 0.0471],
          [0.0353, 0.0353, 0.0392,  ..., 0.0588, 0.0627, 0.0667],
          [0.0706, 0.0549, 0.0431,  ..., 0.0588, 0.0627, 0.0667],
          ...,
          [0.8667, 0.8118, 0.8510,  ..., 0.6667, 0.6941, 0.5804],
          [0.7961, 0.8471, 0.8627,  ..., 0.6353, 0.6745, 0.6000],
          [0.7843, 0.8824, 0.8667,  ..., 0.6118, 0.6902, 0.6353]],
 
         [[0.0549, 0.0667, 0.0902,  ..., 0.0392, 0.0392, 0.0392],
          [0.0627, 0.0627, 0.0667,  ..., 0.0510, 0.0549, 0.0588],
          [0.0980, 0.0824, 0.0706,  ..., 0.0510, 0.0549, 0.0588],
          ...,
          [1.0000, 0.9608, 0.9922,  ..., 0.5569, 0.5804, 0.4627],
          [0.9490, 1.0000, 1.0000,  ..., 0.5176, 0.5569, 0.4824],
          [0.9451, 1.0000, 1.0000,  ..., 0.4902, 0.5686, 0.5137]],
 
         [[0.0941, 0.1059, 0.1294,  ..., 0.0510, 0.0510, 0.0510],
          [0.1020, 0.1020, 0.1059,  ..., 0.0627, 0.0667, 0.0706],
          [0.1373, 0.1216, 0.1098,  ...,

In [4]:
batch_size = 1

train_dataloader = DataLoader(training_data, batch_size=batch_size,shuffle=True)
test_dataloader = DataLoader(testing_data, batch_size=batch_size)

In [5]:
i = 0
for X, y in train_dataloader:
  i = i + 1
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print("bbox", y['bbox'])
  if i > 10:
    break

Shape of X [N, C, H, W]: torch.Size([1, 3, 768, 1024])
bbox tensor([[[464, 202,  54,  76],
         [504, 510, 138, 184],
         [890, 478, 134, 190],
         [292, 228,  96, 114],
         [132, 132,  98, 132],
         [  8, 274,  94, 132],
         [584, 120, 100, 126],
         [372, 452, 122, 144],
         [446, 432,  72, 138]]])
Shape of X [N, C, H, W]: torch.Size([1, 3, 768, 1024])
bbox tensor([[[917, 255,  33,  36],
         [891, 253,  28,  34],
         [623, 216,  26,  36],
         [360, 147,  43,  49],
         [ 29, 210,  25,  34],
         [ 72, 244,  23,  28],
         [216, 301,   8,  11],
         [157, 280,  11,  14],
         [251, 313,   9,  10],
         [238, 306,   7,   8],
         [369, 316,   6,   7],
         [437, 316,   6,   8],
         [612, 326,   7,   7],
         [595, 328,   6,   6],
         [577, 321,   7,   8],
         [540, 308,   6,   9],
         [509, 320,   6,   6],
         [762, 337,   7,   7],
         [782, 337,   6,   8]]])
Shape of

In [16]:
class YOLO_V1(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, 7, 2),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d((2, 2), 2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 192, (3, 3)),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d((2, 2), 2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(192, 128, (1, 1)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(128, 256, (3, 3)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(256, 256, (1, 1)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(256, 512, (3, 3)),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d((2, 2), 2)
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(512, 256, (1, 1)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(256, 512, (3, 3)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(512, 256, (1, 1)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(256, 512, (3, 3)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(512, 256, (1, 1)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(256, 512, (3, 3)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(512, 256, (1, 1)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(256, 512, (3, 3)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(512, 512, (1, 1)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(512, 1024, (3, 3)),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d((2, 2), 2)
        )
        self.layer5 = nn.Sequential(
            nn.Conv2d(1024, 512, (1, 1)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(512, 1024, (3, 3)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(1024, 512, (1, 1)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(512, 1024, (3, 3)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(1024, 1024, (3, 3), 2),
            nn.LeakyReLU(0.1),
        )
        self.layer6 = nn.Sequential(
            nn.Conv2d(1024, 1024, (3, 3), padding='same'),
            nn.LeakyReLU(0.1),
            nn.Conv2d(1024, 1024, (3, 3),padding='same'),
            nn.LeakyReLU(0.1),
        )
        self.layer7 = nn.Sequential(
            nn.Conv2d(1024, 4096, (7, 7)),
            nn.LeakyReLU(0.1),
            nn.Conv2d(4096, 245, (1, 1)),
            nn.LeakyReLU(0.1)
        )
    def forward(self, x):
        f = 0
        if len(x.shape) > 3:
          f = x.shape[0]
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)

        self.layer7 = nn.Sequential(
            nn.Conv2d(1024, 4096, (x.shape[-2], x.shape[-1])),
            nn.LeakyReLU(0.1),
            nn.Conv2d(4096, 245, (1, 1)),
            nn.LeakyReLU(0.1)
        )

        self.layer7 = self.layer7.to(device)

        x = self.layer7(x)
        return torch.reshape(x, (7, 7, 5)) if f == 0 else torch.reshape(x, (f, 7, 7, 5))


In [7]:
import math
lambda_coord = 5
lambda_noobj = .5

In [8]:
def squareDiff(x:tuple, y:tuple):
    t = x - y
    return t[0] ** 2 + t[1] ** 2

def objInCell(box, truth_box):
    print(truth_box.shape)
    center_x = truth_box[0] + truth_box[2] / 2
    center_y = truth_box[1] + truth_box[3] / 2
    # center_x = center_x.numpy()
    # center_y = center_y.numpy()
    return box[0] <= center_x and box[0] + box[2] >= center_x \
            and box[1] <= center_y and box[1] + box[3] >= center_y

In [9]:
S = 7

def Loss(X, bbox):
    loss_coord = 0
    loss_noobj = 0
    loss_0 = 0
    for x in X:
        for i in range(x.shape[0]):
            for j in range(x.shape[1]):
                center_x = x[i, j][0] - x[i, j][2] / 2
                center_y = x[i, j][1] - x[i, j][3] / 2
                # print(x)
                for i in range(bbox.shape[0]):
                    y = bbox[i][0]
                    print("bbox",bbox)
                    print("bbox y ", y)
                    if objInCell((center_x, 
                                center_y,
                                x[i, j][2],x[i, j][3]
                                ), y):
                        loss_coord = loss_coord + squareDiff((center_x, center_y), (y[0], y[1]))
                        loss_coord = loss_coord + squareDiff((math.sqrt(x[i, j][2]),
                                                            math.sqrt(x[i, j][3])), 
                                                            (math.sqrt(y[2]),
                                                            math.sqrt(y[3])))
                        loss_0 = loss_0 + (1 - x[i, j, 4]) ** 2
                    else:
                        loss_noobj = loss_noobj + x[i, j, 4] ** 2

    return lambda_coord * loss_coord + lambda_noobj * loss_noobj + loss_0



In [18]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [19]:
model = YOLO_V1().to(device)
print(model)

YOLO_V1(
  (layer1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2))
    (1): LeakyReLU(negative_slope=0.1)
    (2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1))
    (1): LeakyReLU(negative_slope=0.1)
    (2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(192, 128, kernel_size=(1, 1), stride=(1, 1))
    (1): LeakyReLU(negative_slope=0.1)
    (2): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
    (3): LeakyReLU(negative_slope=0.1)
    (4): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (5): LeakyReLU(negative_slope=0.1)
    (6): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1))
    (7): LeakyReLU(negative_slope=0.1)
    (8): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer4): Sequential(
    (0): Conv2d(512,

In [12]:
for batch, (X, y) in enumerate(train_dataloader):
  print(y)
  break

{'bbox': tensor([[[602, 212, 120, 136]]]), 'blur': tensor([[1]]), 'expression': tensor([[0]]), 'illumination': tensor([[0]]), 'occlusion': tensor([[0]]), 'pose': tensor([[0]]), 'invalid': tensor([[0]])}


In [13]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [14]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        y = y['bbox']
        X, y = X.to(device), y.to(device)
        # print(X.shape)
        # Compute prediction error
        pred = model(X)
        # print(pred.shape)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [20]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, Loss, optimizer)
    # test(test_dataloader, model, loss_fn)
print("Done!")

流式输出内容被截断，只能显示最后 5000 行内容。
         [ 370,  170,    9,   13],
         [ 324,  156,   11,   15],
         [ 287,  169,   10,   11],
         [ 257,  161,   11,   14],
         [ 194,  179,    9,   12],
         [ 178,  173,   11,   11],
         [ 145,  175,   10,   10],
         [ 182,  225,   10,   13],
         [ 219,  176,   10,   14],
         [ 314,  226,   10,   13],
         [ 251,  231,    8,   13],
         [ 269,  245,   11,   15],
         [ 137,  267,   11,   12],
         [  63,  166,   10,   12],
         [  43,  197,   10,   12],
         [ 104,  139,   10,   11],
         [  36,  168,   11,   12],
         [  14,  164,   12,   11],
         [   0,  197,    8,   11],
         [  58,   84,   11,   13],
         [ 604,  444,   15,   17],
         [ 558,  454,   15,   19],
         [  70,  361,   16,   22],
         [  73,  319,   15,   19],
         [ 660,  279,    9,   14],
         [ 488,  235,    7,   11],
         [ 391,  259,   10,   12],
         [ 352,  255,    9, 

OutOfMemoryError: ignored